In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import Data

In [2]:
Data=pd.read_csv('cluster_data_usb.csv')
Data=Data.drop('Unnamed: 0',axis=1)
print(Data.head())
print(Data.shape)

        prd int_tick     spr  rat          rind      cvol   dur      sret  \
0  20090903      U_A  0.0388  BBB  auto_tech_HG  0.035545  6.08 -0.000766   
1  20090904      U_A  0.0388  BBB  auto_tech_HG  0.035545  6.07 -0.001917   
2  20090907      U_A  0.0389  BBB  auto_tech_HG  0.035545  6.06 -0.000107   
3  20090908      U_A  0.0389  BBB  auto_tech_HG  0.035545  6.05 -0.002233   
4  20090909      U_A  0.0391  BBB  auto_tech_HG  0.035545  6.04  0.007124   

       tret  xoil          mcap    beta          debt      iret    trisk  \
0 -0.000060   0.0  8.683420e+09  1.0261  2.125000e+09  0.008535  37.8971   
1 -0.000380   0.0  8.879130e+09  1.0299  2.125000e+09  0.013117  37.9396   
2  0.000180   0.0  8.879130e+09  1.0332  2.125000e+09  0.000000  38.1543   
3 -0.000781   0.0  9.114410e+09  1.0364  2.125000e+09  0.008845  38.0290   
4  0.008814   0.0  9.369790e+09  1.0414  2.125000e+09  0.007782  38.0636   

        sov   ind       lev          size  
0  0.002411  Elec  0.196606  1.08084

# Keep one year

In [3]:
Data['prd'] = Data['prd'].apply(str)
year=Data[Data.prd.str.contains('^2009')]

In [5]:
int_tick_name=list(set(year.int_tick))

# Fill missing prd

In [27]:
# περνω τις ημερομηνιες:
periods=(year.loc[year['int_tick']=='U_TELEFO']).prd

In [29]:
len_int_tick=(year.int_tick).value_counts()
int_tick=pd.Series(len_int_tick.index)
counts=pd.Series(list(len_int_tick))
lenOfInt=pd.concat([int_tick,counts],axis=1)
column_names=['int_tick','count']
lenOfInt.columns=column_names

In [30]:
less=lenOfInt.loc[lenOfInt['count']<81]
less['count']=81-less['count']

C:\Users\thela\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
filled_data=[]
for int_name, missing_num in zip(less['int_tick'],less['count']):
    data_of_missing=year.loc[year['int_tick']==int_name] # To int_tick που του λειπουν μερες
    for_filling=data_of_missing.tail(1)                  # Η τελευταια μερα που θα χρησιμοποιησουμε για την συμπληρωση
    missing=pd.concat([for_filling]*missing_num,axis=0)  # Το τμημα που ελειπε
    filled=pd.concat([data_of_missing,missing],axis=0)   # ενωση του τμηματος που υπαρχει με το υπολοιπον
    filled.prd=list(periods)                             # Ανανεωση των περιοδων
    filled_data.append(filled)
    

In [32]:
filled_data=pd.concat(filled_data)
full=year[~(year["int_tick"].isin(less['int_tick']))] 

In [33]:
DataFill=pd.concat([full,filled_data])

In [34]:
DataFill=DataFill.reset_index(drop=True)

# Sort dataset by period

In [37]:
sorted_=[]
for prd in periods:
    sorted_.append(DataFill.loc[DataFill['prd']==prd])
data=pd.concat(sorted_)

# Rest preprosesing

# rat to numeric

In [38]:
Rating=data.rat

In [39]:
NewRating=[]
for i in Rating:
    if i=="AAA":
        NewRating.append(1)
    if i=="AA":
        NewRating.append(2)
    if i=="A":
        NewRating.append(3)
    if i=="BBB":
        NewRating.append(4)
    if i=="BB":
        NewRating.append(5)
    if i=="B":
        NewRating.append(9)
    if i=="CCC":
        NewRating.append(7)
    if i=="CC":
        NewRating.append(8)
    if i=="C":
        NewRating.append(9)
    if i=="D":
        NewRating.append(10)

In [40]:
NewRating=np.array(NewRating)
NewRating=NewRating/NewRating.max()
NewRating=pd.Series(NewRating)
data.rat=NewRating

# dur to [0-1]

In [41]:
int_data_matrix=[]
for prd in periods:
    int_data=data.loc[data['prd']==prd]
    dur=np.array(int_data.dur)
    dur=(dur-dur.min())/(dur.max()-dur.min())
    int_data.dur=dur
    int_data_matrix.append(int_data)

C:\Users\thela\AppData\Roaming\Python\Python37\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [42]:
data=pd.concat(int_data_matrix)

# Breakpoint lev

In [43]:
lev=data.lev
x=0.2
New_lev=[]
for i in lev:
    if i<0.5:
        New_lev.append(1*x)
    if i>=0.5 and i<0.7:
        New_lev.append(2*x)
    if i>=0.7 and i<0.85:
        New_lev.append(3*x)
    if i>=0.85:
        New_lev.append(4*x)

In [44]:
data.lev=New_lev

# Breakpoint size

In [45]:
size=data['size']
x=0.2
New_size=[]
for i in size:
    if i<2e+9:
        New_size.append(1*x)
    if i>=2e+9 and i<8e+9:
        New_size.append(2*x)
    if i>=8e+9 and i<15e9:
        New_size.append(3*x)
    if i>=15e+9:
        New_size.append(4*x)

In [46]:
data.size=New_size

# Quantile

In [47]:
def add_pctls(ds_in, val_col, num_pctl, by_col = None, new_col = None):
    #print("Running add pctl, params:%s,%s,%s,%s"%(val_col,num_pctl, by_col, new_col));
    new_col = new_col if (new_col) else val_col + "_pctl" ;
    if (by_col is None):
        ds_in.loc[:,new_col]=np.floor(ds_in[val_col].rank() * num_pctl / (len(ds_in[val_col])+1));
    else:
        groupby = ds_in.groupby(by_col)[val_col];
        labels = groupby.ngroup();
        ds_in.loc[:,new_col] = np.floor(groupby.rank() * num_pctl / (groupby.size().values[labels] + 1));
    return ds_in;

In [48]:
def make_quintiles(data,feature):
    prd_sorted=data.prd
    feat=data[feature]
    use_case=pd.concat([feat,prd_sorted],axis=1)
    f_qnt=add_pctls(use_case,feature,100,'prd')
    f_qnt=f_qnt[feature+'_pctl']
    return f_qnt

In [49]:
# spr
spr=make_quintiles(data,'spr')/100
data['spr']=spr

# cvol
cvol=make_quintiles(data,'cvol')/100
data['cvol']=cvol

# xoil
cvol=make_quintiles(data,'xoil')/100
data['xoil']=cvol

# beta
cvol=make_quintiles(data,'beta')/100
data['beta']=cvol

# iret
cvol=make_quintiles(data,'iret')/100
data['iret']=cvol

# trisk
cvol=make_quintiles(data,'trisk')/100
data['trisk']=cvol

# sov
cvol=make_quintiles(data,'sov')/100
data['sov']=cvol

# Exponetial weighting moving averages


In [54]:
# sort by int_tick
int_tick_names_2=list(set(data.int_tick))

In [56]:
data_final=[]
for int_name in int_tick_names_2:
    ticket=data.loc[data['int_tick']==int_name]
    cols = ticket.select_dtypes(include=np.number).columns
    ticket[cols] = ticket[cols].ewm(span=10).mean()
    data_final.append(ticket)

data_final=pd.concat(data_final)

C:\Users\thela\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


# Save

In [58]:
data_final.to_csv('Preproccesed.csv')